In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from dateutil.relativedelta import relativedelta
import config
import datetime
import os
import pickle
import pyperclip
import time

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'
LOGIN_URL = 'https://x.com/login'
#EXECUTABLE_PATH = config.EXECUTABLE_PATH

# Chromeオプションの設定
chrome_options = Options()
chrome_options.add_argument('--disable-popup-blocking')  # ポップアップブロック
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--user-agent=%s' % USER_AGENT)
chrome_options.add_argument('--window-size=1920,1080')

pyperclip.set_clipboard('xclip')

In [3]:
def login():
    global LOGIN_URL

    username = input('usernameを入力してください:')
    password = input('passwordを入力してください:')

    driver = webdriver.Chrome(options=chrome_options)

    #xのログイン画面にアクセス
    driver.get(LOGIN_URL)

    #名前入力画面が出現するまでロードを待つ
    print('入力画面待機中です')
    try:
        WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.TAG_NAME, 'input'))
        ).send_keys(f'{username}')
        driver.find_element(By.XPATH, '//button[2]').click()
    except Exception:
        print('入力画面をロード中にエラーが発生しました')
        driver.save_screenshot('username-input-connection-errors.png')

    #メールアドレス確認画面が出現するまでロードを待つ
    print('メールアドレス確認画面待機中です')
    try:
        mailadress_input = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.TAG_NAME, 'input'))
        )
        mailadress = input('メールアドレスを入力してください：')
        mailadress_input.send_keys(mailadress)
        mailadress_input.send_keys(Keys.ENTER)
    except Exception:
        print('パスワード画面に遷移します')
        pass

    #パスワード入力画面が出現するまでロードを待つ
    print('パスワード画面待機中です')
    try:
        password_input = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.NAME, 'password'))
        )
        password_input.send_keys(f'{password}')
        password_input.send_keys(Keys.ENTER)
    except Exception:
        print('パスワード画面をロード中にエラーが発生しました')
        driver.save_screenshot('password-connection-errors.png')

    #cookieを取得して保存する
    time.sleep(10)
    cookies = driver.get_cookies()

    with open('cookies', 'wb') as f:
        pickle.dump(cookies, f)

    print('ログインしました')
    driver.quit()

In [4]:
def connect_X(driver, url):
    global cookies

    #xのアカウント画面にアクセス
    driver.get(url)

    #cookieを設定して再度アクセス
    for c in cookies:
        driver.add_cookie(c)
    driver.get(url)

    #投稿内容が出現するまでロードを待つ
    print('待機中です')
    try:
        wait = WebDriverWait(driver, 30)
        wait.until(EC.presence_of_all_elements_located)
    except Exception:
        print('ロード中にエラーが発生しました')
        driver.save_screenshot('connection-errors.png')
    
    #body要素を取得
    body = driver.find_element(By.ID, 'react-root')

    return body

In [5]:
def get_tweet_id(driver, dom):
    driver.set_permissions('clipboard-write', 'granted')
    driver.set_permissions('clipboard-read', 'granted')

    #リンクをコピーする
    dom.find_elements(By.TAG_NAME, 'button')[-1].click()
    time.sleep(1)
    driver.find_elements(By.CSS_SELECTOR, "[role='menuitem']")[0].click()
    time.sleep(2)

    #コピーしたリンクを返す
    return driver.execute_script('return window.navigator.clipboard.readText()')

In [61]:
def fetch_posts(driver, url, i: int = 1):
    global target_date

    body = connect_X(driver, url)
    
    post_list = []

    #スクロールさせる幅(初めは0)
    y = 0

    #今日の日付からdateで渡された日数分の投稿を取得
    while True:
        print(f'index:{i} の投稿を取得します')

        #投稿の中に埋め込まれているtimeタグを取得
        try:
            WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, f'//section/div/div/div[{i}]'))
                )
            post_dom = body.find_element(By.XPATH, f'//section/div/div/div[{i}]')
            post_text = post_dom.text.splitlines()
            
        #取得できない場合はスクロールを行なってdom上に表示させる
        except Exception:
            print(f'index:{i} の投稿はdom上に存在しません')

            #スクロール先のy座標
            y += 100
            print(y)
            # スクロールして投稿を読み込むまで待機
            driver.execute_script(f'window.scrollTo({{top: {y}, behavior: "smooth",}});') 
            

            print('投稿を読み込んでいます')
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, f'//section/div/div/div[{i}]'))
            )
            continue

        #投稿の中のtimeタグを取得してtarget_dateよりも新しい日付か確認する
        try:
            time = post_dom.find_element(By.TAG_NAME, 'time').get_attribute('datetime')
            
            s_format = '%Y-%m-%dT%H:%M:%S.000Z'
            dt_time = datetime.datetime.strptime(time, s_format)

        #取得できない場合は投稿以外の要素を取得しているためループを回す
        except:
            i += 1
            continue
        break
    style = post_dom.get_attribute('style')
    return style

    #     #ピン留された投稿、もしくはリポストの場合はループを回す
    #     if post_text[0] == 'Pinned' or 'reposted' in post_text[0]:
    #         i += 1
    #         continue
    #     #新しい投稿の場合はpost_listに内容を追加する
    #     elif target_date < dt_time:
    #         link = get_tweet_id(driver, post_dom)

    #         post_list.append({
    #             'content': post_text,
    #             'link': link
    #             })
    #         i += 1
    #         continue
        
    #     #投稿は日付順になっており、target_dateよりも古い場合これ以上新しい投稿はないと判断してループから抜ける
    #     else:
    #         print('指定された期間の投稿の取得に成功しました。')
    #         driver.close()
    #         break
            
    # return post_list

In [44]:
def calculate_target_date(mode: str, number_of_date: str):
    try:
        int_number_of_date = int(number_of_date)
    except:
        #入力された日数が数字でないためエラー発生
        #再度入力をもらい再帰実行
        mode, period = input('''入力をやり直してください。\n
                             遡る期間数は数字で入力してください。:''').split()
        calculate_target_date(mode, period)

    today = datetime.datetime.today()
    
    match mode:
        #日数で遡る
        case 'd' | 'D':
            target_date = today - datetime.timedelta(days=int_number_of_date)
            return target_date
        
        #週数で遡る
        case 'w' | 'W':
            target_date = today - datetime.timedelta(weeks=int_number_of_date)
            return target_date
        
        #月数で遡る
        case 'm' | 'M':
            target_date = today - relativedelta(months=int_number_of_date)
            return target_date
        
        #いずれの文字にもマッチしない場合は再度入力を受け取る
        case _:
            mode, period = input('''入力をやり直してください。\n
                             遡る間隔は日数(D)、週数(W)、月数(M)から選び、アルファベットで入力してください:''').split()
            calculate_target_date(mode, period)

In [45]:
#cookiesファイルがなければログイン関数を実行してcookieを取得する
if os.path.isfile('cookies') != True:
    cookies = login()
#ファイルがあればcookieを読み込む
else:
    with open ('cookies', 'rb') as f:
        cookies = pickle.load(f)

In [62]:
driver = webdriver.Chrome(options=chrome_options)
driver.execute_script('const newProto = navigator.__proto__;delete newProto.webdriver;navigator.__proto__ = newProto;')

# 指定した要素が見つかるまでの待ち時間を設定する 今回は最大10秒待機する
driver.implicitly_wait(10)

#検索するアカウント
target = 'HAL_Laboratory'

url = f'https://x.com/{target}/'

In [63]:
mode, period = input('''遡る間隔と遡る期間数を入力してください。\n
                     間隔は日数(D)、週数(W)、月数(M)で遡ることができます。\n
                     入力例(日数で遡って、最新から2日前までの投稿を取得する場合):D 2''').split()    

target_date = calculate_target_date(mode, period)
post_list = fetch_posts(driver, url)

待機中です
index:1 の投稿を取得します
index:2 の投稿を取得します


In [64]:
post_list

'transform: translateY(500px); position: absolute; width: 100%;'

In [103]:
y = 0
print(f'window.scrollTo({{top: {y}, behavior: "smooth",}});')

window.scrollTo({top: 0, behavior: "smooth",});
